- Check ONNX Model path
- Prepare Test Data
    - 현재 코드는 학습 코드에서 사용하였던 전처리 기법을 수행한 test 데이터를 기반으로 작성되었기 때문에, train 코드에서 사용하였던 전처리 함수들을 가져와서 전처리 수행 후 test dataset 구축하여 사용 필요

In [ ]:
import os
import numpy as np
import onnxruntime as ort
from sklearn.metrics import classification_report
from tqdm import tqdm

# Load ONNX model
onnx_model_path = "Mel_1s_vit.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Function to make prediction using ONNX model
def predict_onnx(input_data):
    # Run inference
    input_name = ort_session.get_inputs()[0].name
    result = ort_session.run(None, {input_name: input_data})

    return result[0][0]  # Return the prediction result for the segment

# Function to process the entire X_test dataset and get predictions
def predict_x_test(X_test):
    predictions = []

    for segment in X_test:
        # Prepare the input
        input_data = segment[np.newaxis, ...].astype(np.float32)  # Shape: [1, height, width, channels]

        # Get prediction for the segment
        prediction = predict_onnx(input_data)
        predictions.append(prediction)

    return predictions

# Assuming X_test and y_test are already defined as per your previous code
# Example usage for inference on X_test
predictions = predict_x_test(X_test)

# Convert probabilities to binary predictions
y_pred = np.array([1 if pred > 0.8 else 0 for pred in predictions])

# Generate classification report
report = classification_report(y_test, y_pred, target_names=["REAL", "FAKE"])
print("Classification Report:\n", report)